# Multi-Step Training Pipeline with HyperDrive


This notebook covers the process of setting up a basic multi-step training pipeline that utilizes hyperparameter tuning within an Azure Machine Learning workspace.

Steps in this notebook include:

- Training Step (Hyperdrive step used for hyperparameter tuning)
- Evaluation Step
- Registration step

This notebook is meant to serve as a guide or template for users who wish to do hyperparameter tuning within more complex multi-step pipelines.

## Import Dependencies

In [ ]:
# Load dotenv extension

%load_ext dotenv
%dotenv

import os
import sys

sys.path.append("./")

from azureml.core import Environment, Workspace, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import (
    Pipeline,
    PipelineData,
    PipelineParameter,
    TrainingOutput,
)
from azureml.pipeline.steps import PythonScriptStep, HyperDriveStep
from azureml.core.runconfig import RunConfiguration

from azureml.train.hyperdrive import (
    GridParameterSampling,
    BanditPolicy,
    HyperDriveConfig,
    PrimaryMetricGoal,
)
from azureml.train.hyperdrive import choice

from src.common.attach_compute import get_compute
from src.common.get_datastores import get_blob_datastore

## Configure Workspace

In [ ]:
# Interactive login
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(force=False)

Create a workspace object from the `config.json` file in the running directory that you can [download from your AzureML](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-environment#workspace) portal:

In [ ]:
# Restore AML workspace from config.json file (can be downloaded through the portal)
ws = Workspace.from_config()
print(
    "Workspace name: " + ws.name,
    "Azure region: " + ws.location,
    "Subscription id: " + ws.subscription_id,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

## Set Compute Target

Here we set a compute target for our training job to run on, using the script helper `get_compute` imported from `attach_compute.py` file. It will either utilize the default compute for the workspace when the `get_default` parameter is True, or take the information provided within `.env` file to create/configure a specified compute. 

In [ ]:
# Set compute target
compute_target = get_compute(
    workspace=ws,
    get_default=True,
    get_default_type="CPU",
    compute_name=os.getenv("AML_CLUSTER_NAME"),
    vm_size=os.getenv("AML_CLUSTER_CPU_SKU"),
    vm_priority=os.environ.get("AML_CLUSTER_PRIORITY"),
    min_nodes=int(os.environ.get("AML_CLUSTER_MIN_NODES")),
    max_nodes=int(os.environ.get("AML_CLUSTER_MAX_NODES")),
    scale_down=int(os.environ.get("AML_CLUSTER_SCALE_DOWN")),
)

## Configure Datastores 

Here we define the storage containers where data will be accessed, using the script helper `get_blob_datastore` imported from `get_datastores.py` file. It will either utilize the default datastore for the workspace when the `get_default` parameter is True, or take the information provided within `.env` file to register a specified datastore. 

In [ ]:
# Create root datastore
root_datastore = get_blob_datastore(
    workspace=ws,
    get_default=True,
    data_store_name=os.getenv("BLOB_DATASTORE_NAME"),
    storage_name=os.getenv("STORAGE_NAME"),
    storage_key=os.getenv("STORAGE_KEY"),
    container_name=os.getenv("STORAGE_CONTAINER"),
)

# Create input and output data reference
root_dir = DataReference(
    datastore=root_datastore, data_reference_name="data_reference", mode="mount"
)

# Create pipeline data to hold model info in intermediate pipeline steps
model_info_dir = PipelineData(
    "model_info_dir",
    datastore=root_datastore,
    output_mode="mount",
    output_overwrite=True,
)

# Hyperdrive specific output (one file only)
model_info_best = PipelineData(
    "model_info_best",
    datastore=root_datastore,
    output_mode="mount",
    training_output=TrainingOutput("Model", model_file="outputs/model/model_info.json"),
)

## Define Pipeline Parameters

In [ ]:
# Just an example how we can use parameters to provide different input folders and values
model_name = PipelineParameter(name="model_name", default_value="basic_model")

## Define Custom Environment 

Create the Azure ML environment that encapsulates dependencies of our training script:

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Specify package dependencies
batch_conda_deps = CondaDependencies.create(
    pip_packages=[
        "azureml==0.2.7",
        "azureml-core==1.15.0",
        "click==7.0",
        "numpy==1.18.5",
        "pandas==1.1.3",
        "Pillow==7.2.0",
        "tqdm==4.61.0",
    ]
)

# Create and set environment
batch_env = Environment(name="train-env")
batch_env.docker.enabled = True

# Set dependencies that will be used within environment (batch_conda_deps is a set of dependencies)
batch_env.python.conda_dependencies = batch_conda_deps

# Set python version that environment will utilize
batch_conda_deps.set_python_version("3.8.6")

# Set up pipeline run configuration and set environment
run_config = RunConfiguration()
run_config.environment = batch_env

## Create train step

The cells in this section configure and instantiate the steps for training the model using HyperDrive, which carry most differences in transforming a standard training pipeline, without hyperparameter tuning, into a one with HyperDrive. 

In fact, all the previous steps will be required, with no change, for a standard pipeline as well. However, the train step for a standard pipeline will be much simpler, based on a single statement using `PythonScriptStep` class. Using HyperDrive, the `PythonScriptStep` is replaced by `HyperDriveStep` class. Running the `HyperDriveStep` requires two configuration steps to be prepared in advance: `run_config` and `hyperdrive_config`. 

We create the `run_config`, named `train_step_config` in this example, as a `ScriptRunConfig` object that specifies the configuration details of our training job, including our training script, environment to use, and the compute target to run on:

In [ ]:
source_directory = "./"

# Create configurations for training step
train_step_config = ScriptRunConfig(
    script="src/pipeline/train.py",
    compute_target=compute_target,
    source_directory=source_directory,
    environment=batch_env,
)

Now we use the `run_config` created above to configure a run using `HyperDriveConfig` class. Before that, we first define the parameter space using grid sampling, which does a simple grid search over all possible values. In this example, we tune threshold, number of epochs, and learning rate as input parameters. 

We may also use an [early termination policy](https://docs.microsoft.com/azure/machine-learning/service/how-to-tune-hyperparameters#specify-an-early-termination-policy) that automatically ends poorly performing runs to improve computational efficiency.

In the `HyperDriveConfig` object, we need to specify the primary metric to be recorded in our training runs, which is `F_1` metric in our example. Notice that this metric needs to be logged in the training script. We also need to tell the service that we are looking to maximize `F_1`. Further, we set the maximum number of training runs to 500, and maximal concurrent jobs to 4, which is the same as the number of nodes in our compute cluster.

In [ ]:
# Initiate sampling method and provide search space
ps = GridParameterSampling(
    {
        "thr": choice(0.05, 0.20),  # (0.05, 0.20, 0.35, 0.50, 0.65, 0.80, 0.95),
        "epoch": choice(1, 20),  # (1, 20, 40, 60, 80, 100),
        "lr": choice(0.0001, 0.001),  # (0.0001, 0.001, 0.01, 0.1, 1, 5),
    }
)

#  Early termination of low-performance runs
early_termination_policy = BanditPolicy(evaluation_interval=5, slack_factor=0.2)

# Configure hyperdrive config
hd_config = HyperDriveConfig(
    run_config=train_step_config,
    hyperparameter_sampling=ps,
    policy=early_termination_policy,
    primary_metric_name="F_1",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=300,
    max_concurrent_runs=4,
)

Now we are ready to use `HyperDriveStep` to run hyperparameter tuning as a step in pipeline:

In [ ]:
# Create a training step using HyperDrive
train_step = HyperDriveStep(
    name="train step",
    hyperdrive_config=hd_config,
    estimator_entry_script_arguments=[
        "--root_dir",
        root_dir,
        "--model_info_dir",
        model_info_dir,
        "--model_info_best",
        model_info_best,
    ],
    inputs=[root_dir],
    outputs=[model_info_dir, model_info_best],
    allow_reuse=False,
)

## Create evaluation step

The following cell configures and instantiates the step for evaluation (also called test) of the trained model. 

In [ ]:
# Create evaluation step to test the trained model
eval_step = PythonScriptStep(
    name="eval step",
    script_name="src/pipeline/evaluate.py",
    arguments=[
        "--root_dir",
        root_dir,
        "--model_info_dir",
        model_info_dir,
        "--model_info_best",
        model_info_best,
    ],
    inputs=[root_dir, model_info_dir, model_info_best],
    outputs=[],
    compute_target=compute_target,
    source_directory=source_directory,
    runconfig=run_config,
    allow_reuse=False,
)

eval_step.run_after(train_step)

## Create registration step

The following cell configures and instantiates the step for registering the model. 

In [ ]:
# Create a step to register the model
register_step = PythonScriptStep(
    name="register step",
    script_name="src/pipeline/register.py",
    arguments=[
        "--root_dir",
        root_dir,
        "--model_name",
        model_name,
        "--model_info_dir",
        model_info_dir,
    ],
    inputs=[root_dir, model_info_dir],
    outputs=[],
    compute_target=compute_target,
    source_directory=source_directory,
    runconfig=run_config,
    allow_reuse=False,
)

register_step.run_after(eval_step)

## Configure and publish pipeline to AML

In [ ]:
# Create pipeline using existing steps
training_pipeline = Pipeline(workspace=ws, steps=[train_step, eval_step, register_step])

# Check if the pipeline is consistent
training_pipeline.validate()

# Publish pipeline
published_pipeline = training_pipeline.publish(
    name="hyperdrive_training_pipeline",
    description="Hyperdrive training pipeline experiment",
)

## Submit and run pipeline in AML

In [ ]:
# Submit the pipeline
pipeline_run = Experiment(ws, "Reza-hyperdrive-training").submit(training_pipeline)
pipeline_run.wait_for_completion()